In [1]:
import sys, os

# adjust based on what Step 1 printed
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

print("Path added:", os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# now try again
from simulation import load_layout, compute_distance_map, spread_fire_and_smoke, in_bounds
print("Successfully imported simulation.py")


Path added: /mnt/c/Users/marti/Uni/Uni 2025/Computational Modelling/Proj/cits4403-evacuation-simulation/src
pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Successfully imported simulation.py


In [2]:
from simulation import load_layout, compute_distance_map, spread_fire_and_smoke, in_bounds
import os, sys, random, matplotlib.pyplot as plt
from simulation import load_layout, compute_distance_map, spread_fire_and_smoke
import pygame

# Fire Evacuation Simulation

This notebook demonstrates our agent-based fire evacuation model developed in python using pygame.
The simulation models how people (agents) move within a building during a fire, incorporating:
- Fire and smoke spread across the grid 
- Heat diffusion and injury progression based on IS0 13571 thresholds 
- Individual panic and speed levels influencing decision-making 
- BFS-based shortest-path evacuation towards exits 

Agents interact with hazards and exits realistically, allowing users to explore how layout design and behaviour influence escape outcomes.

## Tutorial: Building and Running Your Own Simulation

This section explains how to construct, save, and load custom building layouts for your simulation.

### 1. Launching the Simulation
Run the program using the main file called simulation.py inside of the src directory and make sure to select a suitable width and height (70 pixels for each works well).


### 2. Building the Layout
Construct your building interactively using keyboard shortcuts + mouse clicks.

| Key | Function |
|:----|:-----------|
| 1 | Place or remove walls (black cells) |
| 2 | Add or remove agents (blue cells) |
| 3 | Add or remove exits (green cells) |
| 4 | Add or remove fire sources (red cells) |

Click anywhere on the grid while in a mode to place/remove elements.  
Walls block movement and heat diffusion, exits allow escape, fires spread and generate smoke over time.


### 3. Adjusting Zoom
Use these keys to resize cells:

- `+` = Zoom in
- `–` = Zoom out

Zooming helps inspect larger grids in detail.

### 4. Running and Resetting the Simulation
- Press Spacebar to start or pause.  
- Press R to reset the grid.  

During runtime, the HUD (top of window) displays:
- Agents inside  
- Exited / injured / fatal counts  
- Elapsed time (s)

### 5. Adjusting Global Agent Settings
Press M to open the *Global Agent Menu* and modify:

- Panic level ↑ / ↓  
- Speed ← / →  
- Number of agents affected Z / X  (This number randomly selects however many agents you choose and changes their speed to the amount you requested)
- Press Enter to apply changes  

This menu lets you quickly test different behavioural conditions. 

### 6. Viewing Agent Information
Click on any agent during a paused simulation to open its info panel.  
Displayed fields include:
- ID, speed, age, panic  
- Health status (healthy / injured / fatal / incapacitated)  
- Temperature and exposure levels  

You are also able to use the same controls explained above to change their panic level or speed individually. 

Press ESC to close or click Remove to delete that agent.


### 7. Saving a Layout
Press S to save your current layout as a JSON file:

The file records:
- Grid width and height  
- Walls, exits, fires, and agent coordinates  

You will see a console message confirming save success.


### 8. Loading a Layout
Press L to load a saved layout. By default the code uses:
```python
grid, agents, exits, fires = load_layout("DenseCorridor_layout.json")

```
If you want to change the layout that is loaded you have to close to application, change the filename in this line, and run the application again then click L.  

Unfortunately there is no file-picker menu yet, you must edit this line manually before running if you want to load a layout.


### 9. Ending the Simulation
The simulation automatically ends when all agents:
- Exit successfully, or
- Become incapacitated.

The timer stops and the final results remain visible on the HUD.


## Code Explanation and Demonstrations

In this section, we walk through the main sections of the code and explain how each part functions.  


### Configuration

At the top of the code constants are defined to control the environment and physics of the simulation.  

- Cell size and FPS determine grid resolution and frame rate.
- Colors are RGB tuples for rendering. 
- Cell types (WALL, FIRE, EXIT) are integer-coded for easier grid manipulation. 
- Fire, smoke, and temperature thresholds are based on ISO 13571, giving realistic timing. 

In [3]:
CELL_SIZE = 20
FPS = 10
FIRE_SPREAD_DELAY = 70
SMOKE_SPREAD_DELAY = 18
SAFE_TEMP_THRESHOLD = 50.0

print(f"Cell size: {CELL_SIZE}px, FPS: {FPS}")
print(f"Fire spreads every {FIRE_SPREAD_DELAY} ticks, smoke every {SMOKE_SPREAD_DELAY}.")
print(f"Agents start taking damage above {SAFE_TEMP_THRESHOLD}°C.")

Cell size: 20px, FPS: 10
Fire spreads every 70 ticks, smoke every 18.
Agents start taking damage above 50.0°C.


### Heat Threshold 

The function get_heat_threshold(temp) determines how long an agent can survive at a given temperature.  
These thresholds are pre-calculated from ISO standards for connective heat injury.  

This is how the function classifies heat exposure at different temperatures:

In [4]:
HEAT_THRESHOLDS = [
    (250, 1, 1, 1),
    (200, 1, 2, 3),
    (180, 2, 3, 5),
    (170, 2, 4, 7),
    (160, 3, 5, 8),
    (150, 3, 7, 10),
    (140, 4, 8, 12),
    (130, 5, 10, 16),
    (120, 8, 16, 24),
    (110, 11, 22, 32),
    (100, 16, 32, 48),
    (90, 26, 52, 78),
    (80, 40, 80, 120),
    (70, 68, 136, 204),
    (60, 120, 240, 360),
    (50, 240, 480, 720)
]

def get_heat_thresholds(temp):
    for threshold_temp, nf_ticks, f_ticks, inc_ticks in HEAT_THRESHOLDS:
        if temp >= threshold_temp:
            return (nf_ticks, f_ticks, inc_ticks)
    return (float('inf'), float('inf'), float('inf'))

# Try changing this temperature value:
temp = 100
nf, f, inc = get_heat_thresholds(temp)
fps = 10
print(f"At {temp}°C → Non-fatal in {nf} ticks ({nf/fps:.1f}s), "
      f"Fatal in {f} ticks ({f/fps:.1f}s), "
      f"Incapacitation in {inc} ticks ({inc/fps:.1f}s).")


At 100°C → Non-fatal in 16 ticks (1.6s), Fatal in 32 ticks (3.2s), Incapacitation in 48 ticks (4.8s).


### Temperature to color conversion  
The temp_to_color(temp) function converts numeric temperature values into visible color gradients.  
- White = ambient temperature
- Yellow/orange = moderate heat
- Red = fire

This function is used for visualizing heat diffusion in the grid. 

In [5]:
import matplotlib.pyplot as plt
import numpy as np

AMBIENT_TEMP = 20.0
FIRE_TEMP = 600.0

def temp_to_color(temp):
    if temp <= AMBIENT_TEMP:
        return (255, 255, 255)
    elif temp >= FIRE_TEMP:
        return (255, 0, 0)
    norm = (temp - AMBIENT_TEMP) / (FIRE_TEMP - AMBIENT_TEMP)
    if norm < 0.33:
        t = norm / 0.33
        r, g, b = 255, 255, int(255 * (1 - t))
    elif norm < 0.66:
        t = (norm - 0.33) / 0.33
        r, g, b = 255, int(255 * (1 - 0.5 * t)), 0
    else:
        t = (norm - 0.66) / 0.34
        r, g, b = 255, int(128 * (1 - t)), 0
    return (r, g, b)



### Fire and smoke spread 
The function spread_fire_and_smoke() updates which cells contain fire or smoke every tick.  
- Fire spreads slower (FIRE_SPREAD_DELAY = 70)
- Smoke spreads faster (SMOKE_SPREAD_DELAY = 18)

We can show this in a small sample grid: 

In [6]:
EMPTY, WALL, EXIT, FIRE, SMOKE = 0, 1, 2, 3, 4

def spread_fire_and_smoke(grid, grid_width, grid_height, tick):
    if tick % FIRE_SPREAD_DELAY == 0:
        new_fire = []
        for y in range(grid_height):
            for x in range(grid_width):
                if grid[y][x] == FIRE:
                    for dx, dy in [(1,0),(-1,0),(0,1),(0,-1)]:
                        nx, ny = x+dx, y+dy
                        if 0 <= nx < grid_width and 0 <= ny < grid_height:
                            if grid[ny][nx] in (EMPTY, SMOKE):
                                new_fire.append((nx, ny))
        for fx, fy in new_fire:
            grid[fy][fx] = FIRE

# Example: start with one fire source
grid = [[EMPTY]*5 for _ in range(5)]
grid[2][2] = FIRE

for t in [0, 70, 140]:
    spread_fire_and_smoke(grid, 5, 5, t)
    print(f"Tick {t}:")
    for row in grid:
        print(row)
    print()


Tick 0:
[0, 0, 0, 0, 0]
[0, 0, 3, 0, 0]
[0, 3, 3, 3, 0]
[0, 0, 3, 0, 0]
[0, 0, 0, 0, 0]

Tick 70:
[0, 0, 3, 0, 0]
[0, 3, 3, 3, 0]
[3, 3, 3, 3, 3]
[0, 3, 3, 3, 0]
[0, 0, 3, 0, 0]

Tick 140:
[0, 3, 3, 3, 0]
[3, 3, 3, 3, 3]
[3, 3, 3, 3, 3]
[3, 3, 3, 3, 3]
[0, 3, 3, 3, 0]



### Distance map 
The function compute_distance_map() calculates the shortest distance from every cell to the nearest exit.  
This is how agents decide where to move during each tick using BFS.

In [13]:
from collections import deque

def compute_distance_map(exits, grid, grid_width, grid_height):
    INF = float("inf")
    dist_map = [[INF]*grid_width for _ in range(grid_height)]
    q = deque(exits)
    for ex, ey in exits:
        dist_map[ey][ex] = 0
    while q:
        x, y = q.popleft()
        for dx, dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < grid_width and 0 <= ny < grid_height:
                if grid[ny][nx] != WALL and dist_map[ny][nx] == INF:
                    dist_map[ny][nx] = dist_map[y][x] + 1
                    q.append((nx, ny))
    return dist_map

# Here is a demo, change the exit cell tuple (4,0) passed into the function to see the values change
grid = [[EMPTY]*5 for _ in range(5)]
grid[0][4] = EXIT
dist = compute_distance_map([(4,0)], grid, 5, 5)

for row in dist:
    print(row)


[4, 3, 2, 1, 0]
[5, 4, 3, 2, 1]
[6, 5, 4, 3, 2]
[7, 6, 5, 4, 3]
[8, 7, 6, 5, 4]


### Saving and loading layouts  
Layouts are stored as JSON files for reloading.  
Below shows the code required for both saving and loading a layout. 

In [14]:
def save_layout(filename, grid, agents, exits, fires):
    layout = {
        "grid_width": len(grid[0]),
        "grid_height": len(grid),
        "walls": [(x,y) for y,row in enumerate(grid) for x,c in enumerate(row) if c == WALL],
        "agents": agents,
        "exits": exits,
        "fires": fires
    }
    with open(filename, "w") as f:
        json.dump(layout, f, indent=2)
    print(f"Layout saved → {filename}")

def load_layout(filename):
    with open(filename) as f:
        layout = json.load(f)
    print(f"Layout loaded → {filename}")
    return layout

### Agent Decision and Movement System  
During the simulation, each tick models a sequence of logical updates controlling how agents move.  

This happens in four stages: 
- Stage 0 - Hazard Exposure & Injury Determination
- Stage 1 - Movement Intent Declaration
- Stage 2 - Same cell conflict resolution
- Stage 3 - Exit capacity conflict resolution
- Stage 4 - State update

We will go through and explain the stages

### Stage 0 
We check the surroundings and cell an agent is in and increment their exposure counters (which track how long an agent has been exposed to smoke/heat/fire).  
These counters determine whether the agent transitions from Healthy --> Injured --> Fatally Injured --> Incapacitated.  

The below code illustrates an example of an agent remaining in a lethal cell with higher heat or smoke levels. 

In [15]:
SAFE_TEMP_THRESHOLD = 50.0
HEALTHY, INJURED, FATALLY_INJURED, INCAPACITATED = 0, 1, 2, 3
SMOKE_THRESHOLD = (2, 3, 4)

agent_health = HEALTHY
smoke_exposure = 0

for tick in range(1, 6):
    smoke_exposure += 1
    if smoke_exposure >= SMOKE_THRESHOLD[2]:
        agent_health = INCAPACITATED
    elif smoke_exposure >= SMOKE_THRESHOLD[1]:
        agent_health = FATALLY_INJURED
    elif smoke_exposure >= SMOKE_THRESHOLD[0]:
        agent_health = INJURED
    print(f"Tick {tick}: Exposure={smoke_exposure}, Health={['Healthy','Injured','Fatal','Incap'][agent_health]}")


Tick 1: Exposure=1, Health=Healthy
Tick 2: Exposure=2, Health=Injured
Tick 3: Exposure=3, Health=Fatal
Tick 4: Exposure=4, Health=Incap
Tick 5: Exposure=5, Health=Incap


### Stage 1 Declaring intended moves
This code is just for visual purposes and should not be run here.  
At this stage, each agent decides what to do next based on their state, and distance from the nearest exit.  
Agents that are incapacitated, waiting from a previous slow movement, or already at an exit are skipped here.  
Others look at their surrounding cells and choose where to move: 
- Low panic means they're more likely to move towards a cell closer an exit.
- High panic may randomly choose a sub-optimal neighbor to simulate confusion/human error.  

Agents that reach an exit are added to 'exit_targets', while those left propose a normal move and are put into 'normal_targets' for the next stage. 


In [ ]:
# Stage 1: each agent declares an intended move
for idx, (ax, ay) in enumerate(agents):
    if idx in incapacitated_idx:
        survivors_idx.add(idx)
        continue

    speed = agent_data.get((ax, ay), {}).get("speed", 1.0)
    wait_ticks = agent_data.get((ax, ay), {}).get("wait_ticks", 
    if wait_ticks > 0:
        agent_data[(ax, ay)]["wait_ticks"] = wait_ticks - 1
        survivors_idx.add(idx)
        contin
    if grid[ay][ax] == EXIT:
        exit_pos = (ax, ay)
        exit_targets[exit_pos].append(idx)
        contin
    if dist_map[ay][ax] == float("inf"):
        survivors_idx.add(idx)
        contin
    cands = []
    for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
        nx, ny = ax + dx, ay + dy
        if in_bounds(nx, ny, grid_width, grid_height):
            if grid[ny][nx] != WALL and (nx, ny) not in occupied:
                cands.append((nx, ny
    if not cands:
        survivors_idx.add(idx)
        contin
    cands.sort(key=lambda p: dist_map[p[1]][p[0]])
    best = cands[0]
    cur_dist = dist_map[ay][a
    panic_lvl = agent_data.get((ax, ay), {}).get("panic", 0)
    panic_prob = panic_lvl / 10
    if len(cands) > 1 and random.random() < panic_prob:
        move = random.choice(cands[1:])
    else:
        if dist_map[best[1]][best[0]] < cur_dist:
            move = best
        else:
            survivors_idx.add(idx)
            contin
    if grid[move[1]][move[0]] == EXIT:
        exit_targets[move].append(idx)
    else:
        normal_targets[move].append(idx)


### Stage 2 Conflict Resolution for normal cells  
Once all agents have declared their intended moves, conflicts occur when multiple agents target the same cell.  
This stage randomly selects one winner to occupy the cell and leaves the rest in 'survivors_idx' meaning they stay still for that tick. 

In [ ]:
# Stage 2: resolve conflicts for normal cells (one winner per cell)
for target, idxs in sorted(normal_targets.items()):
    if len(idxs) == 1:
        winners_move[idxs[0]] = target
    else:
        random.shuffle(idxs)
        winners_move[idxs[0]] = target
        for loser in idxs[1:]:
            survivors_idx.add(loser) 


### Stage 3 Exit Resolution (Queuing at doors)  
Exits have a limited capacity 'EXIT_CAPACITY_PER_TICK' so if multiple agents reach an exit simultaneously,  
only a certain number can actually leave per tick.  
The rest remain queued at the door until the next tick.  
'removed_idx' stores agents that successfully exited this tick so they can be removed in the next stage. 

In [ ]:
# Stage 3: resolve exits with capacity (queueing at doors)
exit_cap_remaining = {e: EXIT_CAPACITY_PER_TICK for e in exits}
for exit_pos, idxs in sorted(exit_targets.items()):
    random.shuffle(idxs)
    cap = exit_cap_remaining.get(exit_pos, EXIT_CAPACITY_PER_TICK)
    winners_to_exit = idxs[:cap]
    removed_idx.update(winners_to_exit)
    exited_count += len(winners_to_exit)
    for winner_idx in winners_to_exit:
        health_state = agent_health.get(agents[winner_idx], HEALTHY)
        if health_state == INJURED:
            exited_injured_count += 1
        elif health_state == FATALLY_INJURED:
            exited_fatally_injured_count += 1
    for loser in idxs[cap:]:
        survivors_idx.add(loser)

### Stage 4 Updated Agents States  
After movement and exits are resolved, the grid updates to reflect new positions and conditions.  
Agents that exited or became incapacitated are removed, others carry their heat, smoke, and health data to their new coordinates.  
This keeps exposure tracking and movement logic consistent between ticks.  
'removed_idx' agents in this set are now deleted from all tracking lists to finalize their exit. 

In [ ]:
# Stage 4: build next agents list + carry exposure and health to new coords
new_agents = []
new_exposure = {}
new_agent_data = {}
new_agent_health = {}
new_heat_exposure_ticks = {}
for idx, pos in enumerate(agents):
    if idx in removed_idx:
        if pos in exposure:
            del exposure[pos]
        if pos in agent_data:
            del agent_data[pos]
        if pos in agent_health:
            del agent_health[pos]
        if pos in heat_exposure_ticks:
            del heat_exposure_ticks[pos]
        continue
    if idx in incapacitated_idx:
        if pos in exposure:
            del exposure[pos]
        if pos in agent_data:
            del agent_data[pos]
        if pos in agent_health:
            del agent_health[pos]
        if pos in heat_exposure_ticks:
            del heat_exposure_ticks[pos]
        incapacitated_count += 1
        contin
    new_pos = winners_move[idx] if idx in winners_move else pos
    new_agents.append(new_pos)
    old_exp = exposure.get(pos, {"smoke": 0, "fire": 0})
    new_exposure[new_pos] = old_exp
    if pos in exposure and pos != new_pos:
        del exposure[po
    old_data = agent_data.get(pos)
    if old_data is not None:
        new_agent_data[new_pos] = old_da
        if pos != new_pos:
           speed = max(0.1, old_data["speed"])
           move_delay = max(0, int(round(1 / speed)) - 1)
           new_agent_data[new_pos]["wait_ticks"] = move_del
        if pos != new_pos and pos in agent_data:
            del agent_data[po
    old_health = agent_health.get(pos, HEALTHY)
    new_agent_health[new_pos] = old_health
    if pos in agent_health and pos != new_pos:
        del agent_health[po
    old_heat_ticks = heat_exposure_ticks.get(pos, 0)
    new_heat_exposure_ticks[new_pos] = old_heat_ticks
    if pos in heat_exposure_ticks and pos != new_pos:
        del heat_exposure_ticks[po
agents = new_agents
exposure = new_exposure
agent_data = new_agent_data
agent_health = new_agent_health
heat_exposure_ticks = new_heat_exposure_ticks

Although we cannot display the full Pygame interface here, these notebook cells allow exploration of the logical and mathematical foundations of the simulation.